### Retrieve wind speed data from DWD's [Open Data Server](https://www.dwd.de/EN/ourservices/opendata/opendata.html).

In [13]:
from bs4 import BeautifulSoup
import requests

#### List files to be downloaded

In [16]:
url = 'https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/hourly/wind/recent/'
ext = 'zip'

def listFD(url, ext=''):
    page = requests.get(url).text
#     print(page)
    soup = BeautifulSoup(page, 'html.parser')
    return [url + '/' + node.get('href') for node in soup.find_all('a') if node.get('href').endswith(ext)]

urls = listFD(url, ext)

#### Download zip files, extract CSVs from them, and merge into dataframe
(Method adopted from [this blog post](https://stereopickle.medium.com/how-to-download-unzip-zip-files-in-python-5f326bb1a829))

In [20]:
from io import BytesIO
import pandas as pd
from zipfile import ZipFile
from urllib.request import urlopen

In [54]:
full_df = pd.DataFrame()

for link in urls:
    # open url
    resp = urlopen(link)
    # read zipfile
    zipfile = ZipFile(BytesIO(resp.read()))
    # get the csv file name
    fname = zipfile.namelist()[-1]
    # convert to pandas dateframe
    df = pd.read_csv(zipfile.open(fname), delimiter=';', dtype=object)
    # close zipfile we don't need
    zipfile.close()
    
    # concatenate dataframes
    full_df = pd.concat([full_df, df], ignore_index=True, sort=False)
    
    # let me know how much it is completed.
    print(f'{fname} Completed')

produkt_ff_stunde_20200817_20200930_00011.txt Completed
produkt_ff_stunde_20200817_20220217_00090.txt Completed
produkt_ff_stunde_20200817_20220217_00096.txt Completed
produkt_ff_stunde_20200817_20220217_00102.txt Completed
produkt_ff_stunde_20200817_20220131_00125.txt Completed
produkt_ff_stunde_20200817_20220217_00161.txt Completed
produkt_ff_stunde_20200817_20220217_00164.txt Completed
produkt_ff_stunde_20200817_20220217_00183.txt Completed
produkt_ff_stunde_20200817_20220217_00197.txt Completed
produkt_ff_stunde_20200817_20220217_00198.txt Completed
produkt_ff_stunde_20200817_20220217_00232.txt Completed
produkt_ff_stunde_20200817_20220217_00282.txt Completed
produkt_ff_stunde_20200817_20220217_00298.txt Completed
produkt_ff_stunde_20200817_20220217_00303.txt Completed
produkt_ff_stunde_20200817_20220217_00342.txt Completed
produkt_ff_stunde_20200817_20220131_00368.txt Completed
produkt_ff_stunde_20200817_20220217_00427.txt Completed
produkt_ff_stunde_20200817_20210505_00430.txt Co

produkt_ff_stunde_20200817_20220217_03534.txt Completed
produkt_ff_stunde_20200817_20220131_03623.txt Completed
produkt_ff_stunde_20200817_20220217_03631.txt Completed
produkt_ff_stunde_20200817_20220131_03639.txt Completed
produkt_ff_stunde_20200817_20220217_03651.txt Completed
produkt_ff_stunde_20200817_20220217_03660.txt Completed
produkt_ff_stunde_20200817_20220217_03668.txt Completed
produkt_ff_stunde_20200817_20220217_03730.txt Completed
produkt_ff_stunde_20200817_20220217_03761.txt Completed
produkt_ff_stunde_20200817_20220217_03811.txt Completed
produkt_ff_stunde_20200817_20220217_03821.txt Completed
produkt_ff_stunde_20200817_20220217_03897.txt Completed
produkt_ff_stunde_20200817_20220217_03905.txt Completed
produkt_ff_stunde_20200817_20220217_03925.txt Completed
produkt_ff_stunde_20200817_20220217_03946.txt Completed
produkt_ff_stunde_20200817_20220217_03987.txt Completed
produkt_ff_stunde_20200817_20220217_04024.txt Completed
produkt_ff_stunde_20200817_20220217_04032.txt Co

In [241]:
df = full_df.copy()
df.shape

(3731307, 6)

In [242]:
df.columns = df.columns.str.strip()

In [243]:
df = df.set_index('STATIONS_ID')

In [244]:
df.iloc[:, [1,2,3]] = df.iloc[:,[1,2,3]].astype('float')
df.iloc[:, [1,2,3]] = df.iloc[:, [1,2,3]].clip(lower=0)

In [245]:
df.iloc[:,0] = pd.to_datetime(df.iloc[:,0], format='%Y%m%d%H')

In [246]:
df = df[(df.MESS_DATUM >= '2022-01-01')]
df.shape

(319673, 5)

In [247]:
df = df.drop(columns=['eor'])

In [248]:
df.to_csv('./wind_speed.csv')